In [ ]:
%pip install fastapi[all] uvicorn python-multipart transformers pydantic tensorflow

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | gpg --import -

!echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list

!sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 0E61D3BBAAEE37FE

!sudo apt-get update

!sudo apt-get install ngrok

In [ ]:
%%writefile app.py

from fastapi import FastAPI, Request
from pydantic import BaseModel
from fastapi.responses import JSONResponse
from fastapi import Depends
import json
from transformers import AutoTokenizer, LlamaForCausalLM, GenerationConfig
import torch
from time import perf_counter
import re

app = FastAPI()

class chef_guru:
    id = "BossBattlar/G97"
    def __init__(self):

        model_name = self.id
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # device="cpu"
        print("[NOTE] GPU avilable") if device=="cuda" else print("[NOTE] Running on CPU")
        print("[GET] model")
        self.model = LlamaForCausalLM.from_pretrained(model_name).to(device)
        print("[SUCCESS] Got Model")
        print("[GET] tokenizer")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        print("[SUCCESS] Got tokenizer")

    def formated_prompt(self,quest):
        return f"<user>user:\n{quest}\n<assistant>assistant:\n"

    def generate_resp(self,user_input):
        prompt = self.formated_prompt(user_input)
        inputs = self.tokenizer([prompt], return_tensors="pt")
        generation_config = GenerationConfig(penalty_alpha=0.6,do_sample=True,
                                        max_new_tokens=1024,pad_token_id=self.tokenizer.eos_token_id)
        start_time = perf_counter()
        inputs = self.tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = self.model.generate(**inputs,generation_config=generation_config)
        output_time = perf_counter() - start_time
        print("Time taken ",round(output_time,2))
        return self.tokenizer.decode(outputs[0],skip_special_tokens=True)


class Req(BaseModel):
    ip: str

KitchenWizard = ''

def in_bet(s, word1, word2):
    parts = s.split(word1, 1)
    if len(parts) < 2:
        return None, None, None
    before = parts[0].strip()
    rest = parts[1].strip()
    parts = rest.split(word2, 1)
    if len(parts) < 2:
        return before, None, rest
    between = parts[0].strip()
    after = parts[1].strip()
    return before, between, after

def parse_resp(data):
    user_data = data.split("<user>")[1].split("<assistant>")[0].strip()
    _,disc,_ = in_bet(data,"<assistant>assistant:","Ingredients:")
    ingredisnt = data.split("Ingredients:")[1].strip()

    ingredients = []
    for line in ingredisnt.splitlines():
        if(line ==  'instructions:'):
            break
        ingredients.append(line)

    instructions = []
    steps = re.split(r'\d+\.\s+', data)
    instructions = [f"{i}. {step.strip()}" for i, step in enumerate(steps) if step.strip()]

    recipe = {
        "disctiption": disc,
        'ingredients': ingredients,
        'instructions': instructions
    }
    return recipe

# @app.on_event("startup")
def startup():
    print("[SERVER] Starting")
    global KitchenWizard
    KitchenWizard = chef_guru()
    print("[HUMAN] Model Build Successfull 😁")
startup()
# @app.on_event("before_server_start")
def before_server_start():
    print("[Chef Guru] Hello World! 🤖")

@app.get("/")
async def home():
    return {"status": "alive"}
@app.get("/chast/bot/{data}")
async def chat(data):
    print(data,type(data))
    data = json.loads(data)
    resp = parse_resp(KitchenWizard.generate_resp(data["data"]))
    print(data,resp)
    return JSONResponse({"message": "Hello World", "data": resp})
@app.post("/chat/bot")
async def chats(req:Req):
    print(req.ip)
    generated = KitchenWizard.generate_resp(req.ip)
    print(generated)
    try:
      resp=parse_resp(generated)
    except:
      resp={
        "discription":"why don't you name it your self",
        'ingredients': [],
        'instructions': []
      }
      _,_,res=in_bet(generated,'<user>user:','<assistant>assistant:')
      for line in res.split('\n'):
          if len(line)<50 :
            resp["ingredients"].append(line)
          else:
            resp["instructions"].append(line)
    return resp
@app.post('/hello')
async def hay(req:Request):
    res = await req.body()
    return json.loads(res)

# @app.on_event("shutdown")

def shutdown():
    print("[SERVER] shutdown")


In [ ]:
!ngrok authtoken 2ijS0LEIR5USBAHspOV28MMoXxT_5z5daMZhtCbSWr7gaoHCB

In [ ]:
import subprocess
import time

from ipywidgets import HTML
from IPython.display import display

t = HTML(
    value="0 Seconds",
    description = 'Server is Starting Up... Elapsed Time:' ,
    style={'description_width': 'initial'},
)
display(t)

flag = True
timer = 0

try:
    subprocess.check_output(['curl',"localhost:8000"])
    flag = False
except:
    get_ipython().system_raw('uvicorn app:app --host 127.0.0.1 --port 8000 > server.log 2>&1 &')

res = ""

while(flag and timer < 600):
  try:
    subprocess.check_output(['curl',"localhost:8000"])
  except:
    time.sleep(1)
    timer+= 1
    t.value = str(timer) + " Seconds"
    pass
  else:
    flag = False

if(timer >= 600):
  print("Error: timed out! took more then 10 minutes :(")
subprocess.check_output(['curl',"localhost:8000"])

In [ ]:
import subprocess
import time
import sys
import json

from IPython import get_ipython
get_ipython().system_raw('ngrok http 8000 &')
time.sleep(1)
curlOut = subprocess.check_output(['curl',"http://localhost:4040/api/tunnels"],universal_newlines=True)
time.sleep(1)
ngrokURL = json.loads(curlOut)['tunnels'][0]['public_url']
%store ngrokURL
print(ngrokURL)

In [ ]:
import requests
%store -r ngrokURL

response = requests.post(ngrokURL+'/chat/bot', json={"ip":"generate a recipe with rice batter, oil, chilli powde"}, headers={"Content-Type": "application/json"})
# response = requests.post(ngrokURL+'/hello', json={"ip":"generate a recipe with rice batter, oil, chilli powder"}, headers={"Content-Type": "application/json"})

if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Request failed with status code:", response.status_code)

In [ ]:
!pkill uvicorn
!pkill ngrok
!echo ok

In [ ]:
def in_bet(s, word1, word2):
    parts = s.split(word1, 1)
    if len(parts) < 2:
        return None, None, None
    before = parts[0].strip()
    rest = parts[1].strip()
    parts = rest.split(word2, 1)
    if len(parts) < 2:
        return before, None, rest
    between = parts[0].strip()
    after = parts[1].strip()
    return before, between, after

In [ ]:
data="""<user>user:\ngenerate a recipe for butterscotch cake\n<assistant>assistant:\n* butterscotch cake\n Ingredients:\n- 1 (2 1/2-pound) loaf Italian semolina bread\n- 1/2 cup quick-set gelatin\n- 4 cups whole milk\n- 4 cups (1 1/2 sticks) unsalted butter\n- 3/4 cup granulated sugar\n- 1/2 cup (1 stick) unsalted butter\n- 2 cups (4 sticks) unsweetened cottage cheese 1/2 tsp vanilla extract\n- 2 large eggs\n- 1 teaspoon vanilla 2 cups all-purpose flour\n- 1/2 teaspoon baking soda\n- 1/2 teaspoon salt\n- 1/2 cup unsalted butter\n- 1/2 cup (1 stick) unsalted butter\n- 1/2 cup granulated sugar\n- 1 egg\n- 1 tsp vanilla extract\n- 1 (10-ounce container) cream cheese\n- 1 tsp vanilla extract\n- 1/2 cup (1 stick) unsalted butter\n- 2 cups (3 sticks) granulated sugar\n- 4 tbsp all-purpose flour\n- 1/2 tsp baking soda\n- 1/4 tsp nutmeg\n- 2 tsp rum\n- 1 tsp caramel coloring\n\ninstructions:\n\n1. prepare the brown sugar/butter mixture. In a heatproof bowl, whisk together 3/4 cup light brown sugar and 1/2 cup water. 2. In another bowl, whisk together 2 cups granulated sugar and 1 egg. Prepare the lining and ruffle. 3. In a medium oven-safe skillet, melt the remaining 2 cups dark brown sugar over low heat, stirring constantly, just until it melted. 4. While continuing to stir, heat the rum in a measuring cup. 5. In a large bowl, stir the melted butter, caramel coloring, vanilla, and cornstarch. Stir with a whisk until mixed. Stir in the warm rum and the cream cheese. 6. Add the butter, brown sugar, cottage cheese, and yeast mixture to the melted butter mixture. Stir until well combined. 7. Mix the dry ingredients (flour, baking soda, nutmeg)and salt in a separate bowl. Stir into the ingredients. 8. Pour the wet mixture into the dry mixture. 9. Use immediately or chill until solid. 10. To bind the cake, pour half of the mixture into a buttered 9x13 inch baking dish. 11. Use the remaining egg whites to form a meringue, letting them sit for a full 15 minutes. Use a wooden spoon to fold in the whipped eggs. 12. Pour the egg whites on top of the cake mixture leaving a small border for wrapping. 13. Bake at 325 degrees for 45-60 minutes or until the cake is lightly golden brown on top. 14. Cool completely before frosting. 15. Top with brown sugar, butter, and cinnamon for a butterscotch topping. Enjoy!"""

In [ ]:
import re

data = """<user>user:\ngenerate a recipe for butterscotch cake\n<assistant>assistant:\n* butterscotch cake\n Ingredients:\n- 1 (2 1/2-pound) loaf Italian semolina bread\n- 1/2 cup quick-set gelatin\n- 4 cups whole milk\n- 4 cups (1 1/2 sticks) unsalted butter\n- 3/4 cup granulated sugar\n- 1/2 cup (1 stick) unsalted butter\n- 2 cups (4 sticks) unsweetened cottage cheese 1/2 tsp vanilla extract\n- 2 large eggs\n- 1 teaspoon vanilla 2 cups all-purpose flour\n- 1/2 teaspoon baking soda\n- 1/2 teaspoon salt\n- 1/2 cup unsalted butter\n- 1/2 cup (1 stick) unsalted butter\n- 1/2 cup granulated sugar\n- 1 egg\n- 1 tsp vanilla extract\n- 1 (10-ounce container) cream cheese\n- 1 tsp vanilla extract\n- 1/2 cup (1 stick) unsalted butter\n- 2 cups (3 sticks) granulated sugar\n- 4 tbsp all-purpose flour\n- 1/2 tsp baking soda\n- 1/4 tsp nutmeg\n- 2 tsp rum\n- 1 tsp caramel coloring\n\ninstructions:\n\n1. prepare the brown sugar/butter mixture. In a heatproof bowl, whisk together 3/4 cup light brown sugar and 1/2 cup water. 2. In another bowl, whisk together 2 cups granulated sugar and 1 egg. Prepare the lining and ruffle. 3. In a medium oven-safe skillet, melt the remaining 2 cups dark brown sugar over low heat, stirring constantly, just until it melted. 4. While continuing to stir, heat the rum in a measuring cup. 5. In a large bowl, stir the melted butter, caramel coloring, vanilla, and cornstarch. Stir with a whisk until mixed. Stir in the warm rum and the cream cheese. 6. Add the butter, brown sugar, cottage cheese, and yeast mixture to the melted butter mixture. Stir until well combined. 7. Mix the dry ingredients (flour, baking soda, nutmeg)and salt in a separate bowl. Stir into the ingredients. 8. Pour the wet mixture into the dry mixture. 9. Use immediately or chill until solid. 10. To bind the cake, pour half of the mixture into a buttered 9x13 inch baking dish. 11. Use the remaining egg whites to form a meringue, letting them sit for a full 15 minutes. Use a wooden spoon to fold in the whipped eggs. 12. Pour the egg whites on top of the cake mixture leaving a small border for wrapping. 13. Bake at 325 degrees for 45-60 minutes or until the cake is lightly golden brown on top. 14. Cool completely before frosting. 15. Top with brown sugar, butter, and cinnamon for a butterscotch topping. Enjoy!"""
user_data = data.split("<user>")[1].split("<assistant>")[0].strip()
_,disc,_ = in_bet(data,"<assistant>assistant:","Ingredients:")
ingredisnt = data.split("Ingredients:")[1].strip()
# instruct = data.split("instructions: ")[1].strip()

ingredients = []
for line in ingredisnt.splitlines():
    if(line ==  'instructions:'):
        break
    ingredients.append(line)

instructions = []
steps = re.split(r'\d+\.\s+', data)
instructions = [f"{i}. {step.strip()}" for i, step in enumerate(steps) if step.strip()]

recipe = {
    "disctiption": disc,
    'ingredients': ingredients,
    'instructions': instructions
}

In [ ]:
recipe

In [ ]:
resp="""<user>user:
generate a recipe with rice batter, oil, chilli powder
<assistant>assistant:
1 cup all-purpose rice flour
1/2 cup brown sugar
1/2 teaspoon cinnamon
1/2 teaspoon salt
1/4 teaspoon ground cumin
1/4 teaspoon ground coriander
1/2 cup vegetable oil
1/2 cup chili powder
1/2 teaspoon smoked paprika
1/2 teaspoon garlic powder
1 lime, juiced
Heat the oil in a wok over high heat. Slowly stir in the rice flour. Cook until the flour is foamy.
Stir in the cumin, coriander, cinnamon, cayenne, chili powder, salt and smoked chili powder. Pour in the lime juice. Stir constantly until the mixture is thick and well blended. Allow to cool slightly.
Whisk in the oils to create a smooth batter.
Serve dumplings in coconut water for dipping.
"""


In [ ]:
res